In [1]:
from pcd_dataset.kitti_dataset import KittiDataset

dataset_path = 'dataset_baby/'
sequence = '00'
image_instances_path = 'pipeline_baby/vfm-labels/sam/00/'
kitti = KittiDataset(dataset_path, sequence, image_instances_path)

In [2]:
start_index = 19
end_index = 23
cam_name = 'cam2'

In [3]:
from services.label_service import get_map_not_zero_in_sphere

R = 11
map, points2instances = get_map_not_zero_in_sphere(kitti, cam_name, start_index, end_index, R, False)

map_size1=498154
map_size2=137163
map_size3=78471


In [4]:
from services.label_service import build_o3d_voxel_pcd

voxel_size = 0.3
voxel_pcd, voxel_points2instances, voxel_trace = build_o3d_voxel_pcd(map, points2instances, start_index, end_index, voxel_size)

In [5]:
voxel_pcd

PointCloud with 1243 points.

In [6]:
import numpy as np

from scipy.spatial.distance import cdist
from utils.distances_utils import sam_label_distance

voxel_points = np.asarray(voxel_pcd.points)
spatial_distance = cdist(voxel_points, voxel_points)

dist_voxel_points, masks = sam_label_distance(voxel_points2instances, spatial_distance, 2, 10)

In [7]:
from utils.distances_utils import remove_isolated_points

dist, points, trace = remove_isolated_points(dist_voxel_points, voxel_points, voxel_trace[2])

In [8]:
from services.normalized_cut_service import normalized_cut

T = 0.4
eigenval = 100
clusters = normalized_cut(dist, np.array([i for i in range(len(points))], dtype=int), T, eigenval)

mcut = 3.1517529387182613e-06
mcut = 8.18461007216765e-06
mcut = 2.3373577999974093e-05
mcut = 2.3611409521847336e-05
mcut = 0.19191832358440306
mcut = 0.75


/Library/Python/3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1600: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "


mcut = 0.002068138662523516
mcut = 0.2850123390178141
mcut = 0.0
mcut = 0.875
mcut = -5.469017882513263e-15
mcut = 2.4939707328431776e-16
mcut = 0.019810602341890922
mcut = 0.622701925679819
mcut = 1.1716148031811172e-08
mcut = 0.0
mcut = 0.0
mcut = 8.167796084930133e-06
mcut = 4.8063217814915976e-05
mcut = 0.0004855113664571204
mcut = 0.00028138219671879734
mcut = 0.1253591589179982
mcut = 0.1558293519910005
mcut = 0.7303799158818359
mcut = 0.8333333333333333
mcut = 0.2793896595445877
mcut = 0.6460164745613852
mcut = 0.0287391216352965
mcut = 0.7239819004524888
mcut = 0.7017543859649122
mcut = 0.9285714285714285
mcut = 1.3073622526150101e-08
mcut = 0.8
mcut = 0.7933996072744631
mcut = 0.8571428571428571
mcut = 0.5583355432908956
mcut = 0.1512327525468259
mcut = 0.00010136618064206408
mcut = 0.058842691543651
mcut = 6.441761085500534e-07
mcut = 9.546023981255167e-07
mcut = 0.00020968109119184758
mcut = 0.0030989075946357774
mcut = 0.013631703099792794
mcut = 0.006627515972595822
mcut =

In [9]:
len(clusters)

57

In [10]:
from utils.pcd_utils import color_pcd_by_clusters_and_voxels

pcd_clustered = color_pcd_by_clusters_and_voxels(map, trace, clusters)

In [11]:
import open3d as o3d

o3d.visualization.draw_geometries([pcd_clustered])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [12]:
executed_id = 2
name = "labeled_clouds_test/clouds_{}_{}_R{}_voxel{}_T{}_eigenval{}_execute_id{}_withoutoutlier_withisolated".format(start_index, end_index, R, voxel_size, T, eigenval, executed_id)
filename = name.replace(".", "l") + ".pcd"

o3d.io.write_point_cloud(filename, pcd_clustered)

True